In [ ]:
import pandas as pd
import copy
import os
import geopandas as gpd

os.chdir("") # ADD THE PATH WITH THE PSAFECHOICE PACKAGE

# Import the shp with the road links
The shp MUST contain the columns: inf, pav, obst and cross. There are specific attributes for these columns.

The infrastructure type (inf) should be in string format. The main categories are:
- "1: Urban road with sidewalk less than 1.5 m wide"
- "2: Urban road with sidewalk more than 1.5 m wide"
- "3: Urban road with cycle lane"
- "4: Shared space"

The pavement condition (pav) should be in string format. The main categories are:
- "0: bad condition"
- "1: good condition"

The obstacles existnse (obst) should be in string format. The main categories are:
- "0: yes obstacles"
- "1: no obstacles'

The "zebra" pedestrian crossing type and existense should be in string format. The main categories are:
- "0: without pedestrian crossings"
- "1: with pedestrian crossings not controlled by traffic lights"
- "2: with pedestrian crossing controlled by traffic lights"

In [ ]:
scenario_athens_url = "" # ADD THE PATH

links =  gpd.read_file(os.path.join(scenario_athens_url,"baseNetworkAthensLinks_v1.shp"))
# links = links.dropna(subset=['inf', 'pav', 'obst', 'cross'])
print(links.columns) # check if the columns are present.
check_cols = ['inf', 'pav', 'obst', 'cross']
for c in check_cols: print(links.groupby(c).size()) # check if the have the previously specified catagories.
# It also provides descriptive statistics about the infrastructure.

The estimation of perceived safety score is performed based on the PsafeChoices packages.
To install the PsafeChoices package (ONLY) please type:
```bash
pip install git+https://github.com/panosgjuras/Perceived_safety_choices
```

# Import the perceived safety model

The perceived safety model contains beta parameters and kappa thresholds.

If there is a constant, it should be eliminated. Practically, it is is the kappa.0 of the model. This update is required if the Rchoice 
package is used to estimate it.

Please check the output table, next functions require this standardized format.

In [ ]:
from calc import coeffUpd # ADD THE PATH
models_path = ""
cf = pd.read_csv(os.path.join(models_path,'psafe','simple_psafe_models.csv'))
cf = cf.rename(columns={'Unnamed: 0': 'coeffs'})
cf = cf.set_index('coeffs')
cf = coeffUpd(cf) # Please check all the coefficients
cf

# Estimation of perceived safety scores

Perceived safety scores (f'LevPsafe{m}') are estimated per link and transport mode.

This requires the calculation of a Latent Variable (f'LatPsafe{m}') first.

The results are saved in the geoDataframe

In [ ]:
from calc import processRowEst

# define transport modes for which perceived safety will be estimated
modes = ['car', 'ebike', 'escoot', 'walk']

for m in modes:
    latent_vars = []
    safety_levels = []
    
    # Iterate through each row of the GeoDataFrame using iterrows()
    for index, row in links.iterrows():        
        latent_vars.append(processRowEst(index, row, modes, cf)[f'LatPsafe{m}'])
        safety_levels.append(processRowEst(index, row, modes, cf)[f'LevPsafe{m}'])

    links[f'LatPsafe{m}'] = latent_vars
    links[f'LevPsafe{m}'] = safety_levels

# Create a perceived safety score map per transport mode

As the plotting functions, do some modifications in the original dataframe, please use deepcopy.

In [ ]:
from mapAnalysis import plotPsafeLev

for m in modes: plotPsafeLev(copy.deepcopy(links), m)

# Estimate the kernel density of safe links per transport mode

In this process, the mid point of each link is defined. So, the KDE is performed based on mid points of links.

As the plotting functions, do some modifications in the original dataframe, please use deepcopy.

In [ ]:
from mapAnalysis import PsafeHeatmaps

for m in modes: PsafeHeatmaps(copy.deepcopy(links), m)